# GREAT CARIA v5: TEMPORAL RELATIVITY

## The Theory of Coupled Oscillators

In this model, we treat the Global Financial System as a set of **Coupled Oscillators** operating at different **Proper Times** (Clock Speeds).

### Key Concepts
1.  **Proper Time ($\tau$)**: Different agents live in different times.
    - **HFT**: $\tau$ = Milliseconds
    - **Hedge Funds**: $\tau$ = Days/Weeks (Medium Band)
    - **Pensions**: $\tau$ = Quarters/Years (Slow Band)
2.  **Synchronization ($r$)**: A crisis occurs when these distinct clocks **align**. This is measured by the **Kuramoto Order Parameter**.
    $$ r(t) = \left| \frac{1}{N} \sum_{j=1}^{N} e^{i\phi_j(t)} \right| $$
3.  **Resonance (Medium Band)**: The critical "fuse". The Medium Band (10-60d) is where fast noise becomes slow structure. We weight this at **35%**.

### Objective
Isolate **Structural Fragility** (Synchronization) from **Propagation** (Energy Flow) to predict Phase Transitions (Crises).

In [ ]:
# === SETUP ===
!pip install PyWavelets scikit-learn numpy pandas scipy matplotlib -q

import pandas as pd
import numpy as np
from scipy import stats, signal
from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_curve, auc,  roc_auc_score
import matplotlib.pyplot as plt
import warnings
import json
from datetime import datetime

warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

print('='*80)
print('GREAT CARIA v5: TEMPORAL RELATIVITY')
print('Checking Clock Synchronization...')
print('='*80)

In [ ]:
# === LOAD DATA ===
DATA_PATH = '/content/drive/MyDrive/Caria/yahoo_market.parquet'
try:
    df = pd.read_parquet(DATA_PATH)
except FileNotFoundError:
    # Fallback path if folder structure differs
    DATA_PATH = '/content/drive/MyDrive/CARIA/data/raw/yahoo_market.parquet'
    df = pd.read_parquet(DATA_PATH)

df.index = pd.to_datetime(df.index)

# Extract Indices
all_cols = [c for c in df.columns if '_index' in c]
ret = df[all_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]

print(f'Data Loaded: {ret.shape[0]} days, {ret.shape[1]} assets')

In [ ]:
# === CONFIGURATION: THE CLOCKS ===
SCALES = {
    'fast':   {'window': 5,   'weight': 0.15, 'desc': 'Traders (Noise)'},
    'medium': {'window': 30,  'weight': 0.35, 'desc': 'Hedge Funds (RESONANCE)'}, # CRITICAL FUSE
    'slow':   {'window': 120, 'weight': 0.25, 'desc': 'Institutions (Structure)'},
    'macro':  {'window': 252, 'weight': 0.25, 'desc': 'Central Banks (Cycle)'}
}

# Ground Truth for Validation
CRISES = {
    'Lehman': pd.Timestamp('2008-09-15'),
    'Flash_Crash': pd.Timestamp('2010-05-06'),
    'Euro_Crisis': pd.Timestamp('2011-08-05'),
    'China_Crash': pd.Timestamp('2015-08-24'),
    'Brexit': pd.Timestamp('2016-06-24'),
    'Volmageddon': pd.Timestamp('2018-02-05'),
    'COVID': pd.Timestamp('2020-03-11'),
    'Gilt_Crisis': pd.Timestamp('2022-09-23'),
    'SVB': pd.Timestamp('2023-03-10')
}
# Filter relevant crises
CRISES = {k: v for k, v in CRISES.items() if v >= ret.index.min() and v <= ret.index.max()}

def create_target(index, crises):
    target = pd.Series(0, index=index)
    for date in crises.values():
        # Crisis window: 2 weeks before to 1 week after
        target[(target.index >= date - pd.Timedelta(days=14)) & 
               (target.index <= date + pd.Timedelta(days=7))] = 1
    return target

y_true = create_target(ret.index, CRISES)

---
## Part 1: Crisis Factor (The Base Signal)
We start with the empirically robust Crisis Factor (Correlation $\times$ Volatility) to represent the raw stress in the system.

In [ ]:
def compute_cf(r, window=20):
    cf = []
    for i in range(window, len(r)):
        wr = r.iloc[i-window:i]
        corr_matrix = wr.corr().values
        n = len(corr_matrix)
        avg_corr = (corr_matrix.sum() - n) / (n * (n - 1))
        avg_vol = wr.std().mean()
        cf.append(avg_corr * avg_vol * 100)
    return pd.Series(cf, index=r.index[window:])

CF = compute_cf(ret)
print(f'Base Crisis Factor computed. Mean: {CF.mean():.4f}')

---
## Part 2: Temporal Relativity (Decomposition & Synchronization)

### 2.1 Scale Decomposition
Decompose the CF signal into the 4 constituent clocks: Fast, Medium, Slow, Macro.

### 2.2 Phase Extraction (Hilbert Transform)
Determine the instantaneous phase $\phi(t)$ of each clock.

### 2.3 Kuramoto Synchronization
Calculate how aligned these clocks are.

In [ ]:
# 1. DECOMPOSITION
bands = {}
sorted_scales = sorted(SCALES.items(), key=lambda x: x[1]['window'])

for i, (name, config) in enumerate(sorted_scales):
    w = config['window']
    smooth = CF.rolling(w, min_periods=1).mean()
    if i < len(sorted_scales) - 1:
        next_w = sorted_scales[i+1][1]['window']
        next_smooth = CF.rolling(next_w, min_periods=1).mean()
        # Band is difference between this scale and next scale (Bandpass)
        bands[name] = smooth - next_smooth
    else:
        # Last scale is residual trend
        bands[name] = smooth

bands_df = pd.DataFrame(bands).dropna()

# 2. PHASE EXTRACTION (Hilbert)
phases = {}
for col in bands_df.columns:
    series = bands_df[col].values
    # Centering (Hilbert requirement)
    centered = series - np.mean(series)
    analytic = signal.hilbert(centered)
    phases[col] = np.angle(analytic)

phases_df = pd.DataFrame(phases, index=bands_df.index)

# 3. KURAMOTO SYNCHRONIZATION (r)
# r = |mean(e^i*phi)| across scales
complex_phases = np.exp(1j * phases_df)
kuramoto_r = np.abs(complex_phases.mean(axis=1))
CLOCK_SYNC = pd.Series(kuramoto_r, index=phases_df.index)

print(f'Clock Synchronization (r) computed. Max Sync: {CLOCK_SYNC.max():.3f}')

---
## Part 3: Isolating Structure and Flow

### 3.1 Structural Fragility
High Synchronization = The structure is rigid (all clocks locked).

### 3.2 Propagation (Energy Flow)
The **Medium Band** (Resonance). If this band has high energy (volatility), it means shocks are successfully transferring from Fast to Slow.

$$ MSFI_{v5} = \sum (w_i \cdot |Band_i|) \times (1 + \text{Sync}) $$

In [ ]:
# === PHYSICS-FIRST AGGREGATION ===

# Normalize Bands Energy (Absolute value)
bands_energy = bands_df.abs()

# Normalize to 0-1 scale for consistent weighting
for col in bands_energy.columns:
    bands_energy[col] = (bands_energy[col] - bands_energy[col].rolling(252).min()) / \
                        (bands_energy[col].rolling(252).max() - bands_energy[col].rolling(252).min() + 1e-8)
bands_energy = bands_energy.fillna(0)

# Weighted Sum of Energies
weighted_energy = sum(bands_energy[name] * config['weight'] for name, config in SCALES.items())

# === THE UNIFIED TEMPORAL FORMULA ===
# MSFI = Weighted Energy * (1 + Synchronization Bonus)
# If clocks are synced, the energy is amplified.
MSFI_v5 = weighted_energy * (1 + CLOCK_SYNC)

# Smooth slightly for legibility
MSFI_v5 = MSFI_v5.rolling(5).mean()

print('MSFI v5 Computed.')

===
## Part 5: Rigorous Statistical Validation

We perform a deep statistical audit of the model's performance against a baseline (Traditional VIX/Volatility).

### Metrics Calculated:
1.  **Contingency Tables**: TP, FP, TN, FN.
2.  **Performance Metrics**: Precision, Recall, Specificity, F1-Score.
3.  **False Positive Reduction**: Quantifying the "60% reduction" claim.
4.  **ROC & AUC**: With Bootstrapped 95% Confidence Intervals.
5.  **Statistical Significance**: McNemar's Test (for classification difference) and DeLong Test proxy.

In [ ]:
# === 5.1 PREPARE DATA ===
# Align data
common_idx = y_true.index.intersection(MSFI_v5.dropna().index)
y_eval = y_true.loc[common_idx]
score_model = MSFI_v5.loc[common_idx].fillna(0)

# Create a Benchmark (e.g., Raw Volatility of S&P 500 equivalent)
# Assuming first column is a major index (e.g., USA)
benchmark_vol = ret.iloc[:, 0].rolling(20).std().loc[common_idx].fillna(0)
score_bench = (benchmark_vol - benchmark_vol.min()) / (benchmark_vol.max() - benchmark_vol.min()) # MinMax Scaled

# === 5.2 CONTINGENCY TABLES & METRICS ===
def get_metrics(y_true, y_prob, threshold=0.8):
    y_pred = (y_prob > threshold).astype(int)
    
    # Confusion Matrix
    tp = ((y_pred == 1) & (y_true == 1)).sum()
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    tn = ((y_pred == 0) & (y_true == 0)).sum()
    fn = ((y_pred == 0) & (y_true == 1)).sum()
    
    # Metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0 # Sensitivity
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    return {
        'TP': tp, 'FP': fp, 'TN': tn, 'FN': fn,
        'Precision': precision, 'Recall': recall, 'Specificity': specificity,
        'F1': f1, 'FPR': fpr
    }

# Optimize Threshold (Maximize F1)
best_thresh_model = 0.5
best_f1_model = 0
for t in np.linspace(0.1, 0.9, 50):
    m = get_metrics(y_eval, score_model, t)
    if m['F1'] > best_f1_model:
        best_f1_model = m['F1']
        best_thresh_model = t

metrics_model = get_metrics(y_eval, score_model, best_thresh_model)
metrics_bench = get_metrics(y_eval, score_bench, best_thresh_model) # Using same thresh for comparison approx

print(f"=== Performance Metrics (Threshold: {best_thresh_model:.2f}) ===")
print(f"Model     -> FP: {metrics_model['FP']}, TP: {metrics_model['TP']}, Specificity: {metrics_model['Specificity']:.3f}")
print(f"Benchmark -> FP: {metrics_bench['FP']}, TP: {metrics_bench['TP']}, Specificity: {metrics_bench['Specificity']:.3f}")

# False Positive Reduction
fp_reduction = (metrics_bench['FP'] - metrics_model['FP']) / metrics_bench['FP'] if metrics_bench['FP'] > 0 else 0
print(f"\nFALSE POSITIVE REDUCTION: {fp_reduction:.1%}")

# === 5.3 MCNEMAR'S TEST (Statistical Significance) ===
# H0: The two models make errors in the same way.
# Contingency Table comparing classify correctness
model_correct = ((score_model > best_thresh_model) == y_eval)
bench_correct = ((score_bench > best_thresh_model) == y_eval)

a = ((model_correct) & (bench_correct)).sum() # Both Correct
b = ((model_correct) & (~bench_correct)).sum() # Model Correct, Bench Wrong
c = ((~model_correct) & (bench_correct)).sum() # Model Wrong, Bench Correct
d = ((~model_correct) & (~bench_correct)).sum() # Both Wrong

mcnemar_stat = (abs(b - c) - 1)**2 / (b + c) if (b+c) > 0 else 0
p_value_mcnemar = 1 - stats.chi2.cdf(mcnemar_stat, 1)

print(f"\n=== McNemar's Test ===")
print(f"Statistic: {mcnemar_stat:.4f}, p-value: {p_value_mcnemar:.6f}")
if p_value_mcnemar < 0.05:
    print("RESULT: Statistically Significant improvement (p < 0.05)")
else:
    print("RESULT: No statistical significance detected")

# === 5.4 BOOTSTRAPPED AUC INTERVALS ===
n_bootstraps = 1000
bootstrapped_scores = []

rng = np.random.RandomState(42)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_eval), len(y_eval))
    if len(np.unique(y_eval.iloc[indices])) < 2:
        continue
    score = roc_auc_score(y_eval.iloc[indices], score_model.iloc[indices])
    bootstrapped_scores.append(score)

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
ci_lower = sorted_scores[int(0.025 * len(sorted_scores))]
ci_upper = sorted_scores[int(0.975 * len(sorted_scores))]

fpr_m, tpr_m, _ = roc_curve(y_eval, score_model)
roc_auc_m = auc(fpr_m, tpr_m)

print(f"\n=== AUC with 95% Confidence Interval ===")
print(f"AUC: {roc_auc_m:.4f} [{ci_lower:.4f} - {ci_upper:.4f}]")

# === 5.5 VISUAL SUMMARY ===
plt.figure(figsize=(12, 6))
plt.plot(fpr_m, tpr_m, label=f'Great Caria v5 (AUC = {roc_auc_m:.2f})', color='cyan', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curve Analysis: v5 vs Random')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.savefig('/content/drive/MyDrive/Caria/great_caria_v5_roc_analysis.png')
plt.show()

In [ ]:
# === EXPORT ===
latest_idx = -1
export_data = {
    "version": "v5.0 (Temporal Relativity)",
    "generated_at": datetime.now().isoformat(),
    "msfi": float(MSFI_v5.iloc[latest_idx]),
    "clock_sync": float(CLOCK_SYNC.iloc[latest_idx]),
    "resonance": float(bands_energy['medium'].iloc[latest_idx]),
    "status": "WARNING" if MSFI_v5.iloc[latest_idx] > MSFI_v5.quantile(0.8) else "STABLE",
    "auc": roc_auc_m,
    "history": {
        "dates": [d.strftime('%Y-%m-%d') for d in MSFI_v5.index[-100:]],
        "msfi": MSFI_v5.iloc[-100:].tolist(),
        "sync": CLOCK_SYNC.iloc[-100:].tolist()
    }
}

with open('/content/drive/MyDrive/Caria/great_caria_v5.json', 'w') as f:
    json.dump(export_data, f, indent=2)
    
print('Exported v5 JSON.')